In [1]:
# 전체로우데이터 전처리 (202501_04_clean)

In [2]:
# =========================
# Namhae PM2.5 Metals: Cleaning vFinal (for MC fitting & LECR)
# 정책: 음수→NaN, IQR 3.0 셀-마스킹, 행 삭제 금지, 전/후 QA 리포트 저장
# =========================

# --- imports & display ---
import pandas as pd
import numpy as np
import io, os, re
from google.colab import files

pd.options.display.float_format = "{:.6g}".format

# =========================
# 설정값 (네오 기준 확정안)
# =========================
CONFIRM_DETECTED_COLUMNS = True

# 음수→NaN
NEGATIVE_TO_NAN = True

# IQR 셀-마스킹 적용 + 완화 기준(IQR_K=3.0)
APPLY_IQR_FILTER = True
IQR_K = 3.0  # Tukey 기본 1.5보다 완화 → 피크 이벤트 보존

# 대상 전부 NaN인 행만 삭제
DROP_EMPTY_TARGET = True

# =========================
# 파일 업로드(Colab 위젯) + 메모리 직접 로드
# =========================
def _safe_basename(name: str) -> str:
    base, ext = os.path.splitext(name)
    base = re.sub(r"[^\w\-]+", "_", base).strip("_")
    return base, ext

print("엑셀 파일(.xlsx) 1개 이상 업로드하세요.")
uploaded = files.upload()
if not uploaded:
    raise RuntimeError("업로드된 파일이 없습니다. 다시 실행하여 업로드하세요.")

xlsx_names = [k for k in uploaded.keys() if k.lower().endswith(".xlsx")]
if not xlsx_names:
    raise RuntimeError("업로드된 파일 중 .xlsx 확장자가 없습니다.")

src_name = xlsx_names[0]
print("Uploaded (in-memory):", src_name)

# 메모리에서 바로 DataFrame 로드
df_raw = pd.read_excel(io.BytesIO(uploaded[src_name]))
df_raw.columns = [str(c).strip() for c in df_raw.columns]

# 원본 파일명 기반 산출물 경로 자동 설정
_base, _ = _safe_basename(src_name)
RAW_REPORT_PATH   = f"{_base}_QA_raw.xlsx"     # 전처리 전 QA
CLEAN_REPORT_PATH = f"{_base}_QA_clean.xlsx"   # 전처리 후 QA
CLEAN_DATA_PATH   = f"{_base}_clean.xlsx"      # 전처리 후 데이터(엑셀)
CLEAN_DATA_CSV    = f"{_base}_clean.csv"       # 전처리 후 데이터(CSV)

print("Output paths set:")
print(" RAW_REPORT_PATH   =", RAW_REPORT_PATH)
print(" CLEAN_REPORT_PATH =", CLEAN_REPORT_PATH)
print(" CLEAN_DATA_PATH   =", CLEAN_DATA_PATH)
print(" CLEAN_DATA_CSV    =", CLEAN_DATA_CSV)

# =========================
# 유틸: 컬럼 자동 검출
# =========================
def detect_columns(df: pd.DataFrame):
    df = df.copy()
    df.columns = [str(c).strip() for c in df.columns]

    # 시간 후보
    time_candidates = [c for c in df.columns
                       if any(k in c.lower() for k in ["pump-begin", "date", "datetime", "time", "측정일시"])]
    time_col = time_candidates[0] if len(time_candidates) > 0 else None

    # PM2.5 후보
    pm25_candidates = [c for c in df.columns
                       if ("pm2.5" in c.lower())
                       or ("con(ug/m3)" in c.lower())
                       or ("ug/m3" in c.lower() and "con" in c.lower())]
    pm25_col = pm25_candidates[0] if len(pm25_candidates) > 0 else None

    # 금속 단위 패턴
    metal_markers = ["(ng/m3)", "(ng/㎥)", "ng/m3", "ng/㎥"]
    metal_cols = [c for c in df.columns if any(m.lower() in c.lower() for m in metal_markers)]

    # 숫자형 후보(참고용)
    numeric_cols = []
    for c in df.columns:
        ser = pd.to_numeric(df[c], errors="coerce")
        if ser.notna().mean() >= 0.7:
            numeric_cols.append(c)

    return {"time_col": time_col, "pm25_col": pm25_col,
            "metal_cols": metal_cols, "numeric_cols": numeric_cols}

# =========================
# 로드 & 자동 검출 결과 출력
# =========================
detected = detect_columns(df_raw)
time_col   = detected["time_col"]
pm25_col   = detected["pm25_col"]
metal_cols = detected["metal_cols"]
numeric_cols = detected["numeric_cols"]

print("Detected time_col:", time_col)
print("Detected pm25_col:", pm25_col)
print("Detected metal_cols (first 10):", metal_cols[:10])
print("Detected numeric_cols (first 10):", numeric_cols[:10])

if not CONFIRM_DETECTED_COLUMNS:
    raise RuntimeError("자동 검출된 컬럼 확인 후, 맞으면 CONFIRM_DETECTED_COLUMNS=True로 설정하세요.")

# =========================
# QA 리포트 함수
# =========================
def summarize_series(s: pd.Series, name: str):
    s_num = pd.to_numeric(s, errors="coerce")
    return pd.Series({
        "col": name,
        "count_total": int(s_num.shape[0]),
        "count_valid": int(s_num.notna().sum()),
        "valid_pct": float(100.0 * s_num.notna().mean()),
        "mean": float(s_num.mean(skipna=True)) if s_num.notna().any() else np.nan,
        "std": float(s_num.std(skipna=True)) if s_num.notna().any() else np.nan,
        "p5": float(s_num.quantile(0.05)) if s_num.notna().any() else np.nan,
        "p50": float(s_num.quantile(0.50)) if s_num.notna().any() else np.nan,
        "p95": float(s_num.quantile(0.95)) if s_num.notna().any() else np.nan,
        "min": float(s_num.min(skipna=True)) if s_num.notna().any() else np.nan,
        "max": float(s_num.max(skipna=True)) if s_num.notna().any() else np.nan,
    })

def build_qc_table(df: pd.DataFrame, targets: list):
    rows = []
    for c in targets:
        if c in df.columns:
            rows.append(summarize_series(df[c], c))
    return pd.DataFrame(rows)

# =========================
# 대상 컬럼 정의
# =========================
target_cols = []
if pm25_col:
    target_cols.append(pm25_col)
if isinstance(metal_cols, (list, tuple)):
    target_cols += list(metal_cols)
target_cols = [c for c in target_cols if c in df_raw.columns]
assert len(target_cols) > 0, "PM2.5 또는 금속 대상 컬럼을 찾지 못했습니다."

# =========================
# QA 리포트(전처리 전)
# =========================
qc_raw = build_qc_table(df_raw, target_cols)
with pd.ExcelWriter(RAW_REPORT_PATH, engine="openpyxl") as w:
    qc_raw.to_excel(w, index=False, sheet_name="RAW_QA")

# =========================
# 전처리
# =========================
df_clean = df_raw.copy()

# 시간 정렬
if (time_col is not None) and (time_col in df_clean.columns):
    df_clean[time_col] = pd.to_datetime(df_clean[time_col], errors="coerce")
    df_clean = df_clean.sort_values(time_col).reset_index(drop=True)

# 대상만 숫자화(별도 프레임 보관)
num_targets = df_clean[target_cols].apply(pd.to_numeric, errors="coerce")

# --- 음수 → NaN ---
neg_before = int((num_targets < 0).sum().sum())
if NEGATIVE_TO_NAN:
    num_targets = num_targets.mask(num_targets < 0, np.nan)

# --- IQR 셀-마스킹 (행 삭제 금지) ---
cells_masked = 0
if APPLY_IQR_FILTER:
    Q1 = num_targets.quantile(0.25)
    Q3 = num_targets.quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - IQR_K * IQR
    upper = Q3 + IQR_K * IQR

    low_mask  = num_targets.lt(lower, axis=1)
    high_mask = num_targets.gt(upper, axis=1)
    outlier_mask = low_mask | high_mask

    cells_masked = int(outlier_mask.sum().sum())
    num_targets = num_targets.mask(outlier_mask, np.nan)

# 대상 컬럼 적용
df_clean[target_cols] = num_targets

# --- 대상 전부 NaN 행 삭제(선택) ---
rows_before_drop_empty = len(df_clean)
if DROP_EMPTY_TARGET:
    df_clean = df_clean.dropna(subset=target_cols, how="all").reset_index(drop=True)
rows_dropped_empty = rows_before_drop_empty - len(df_clean)

neg_after = int((df_clean[target_cols].apply(pd.to_numeric, errors="coerce") < 0).sum().sum())

# =========================
# 요약 출력
# =========================
print("=== Cleaning Summary ===")
print(f"Target columns (n={len(target_cols)}): {target_cols[:6]}{' ...' if len(target_cols)>6 else ''}")
print(f"Negatives (before -> after): {neg_before} -> {neg_after}")
print(f"IQR masked cells (k={IQR_K}): {cells_masked}")
print(f"Empty-target dropped rows: {rows_dropped_empty}")
print(f"Shape: raw {df_raw.shape} -> clean {df_clean.shape}")

# =========================
# QA 리포트(전처리 후)
# =========================
qc_clean = build_qc_table(df_clean, target_cols)
with pd.ExcelWriter(CLEAN_REPORT_PATH, engine="openpyxl") as w:
    qc_clean.to_excel(w, index=False, sheet_name="CLEAN_QA")

# =========================
# 저장 + 다운로드
# =========================
df_clean.to_excel(CLEAN_DATA_PATH, index=False)
df_clean.to_csv(CLEAN_DATA_CSV, index=False)

files.download(CLEAN_DATA_PATH)
files.download(CLEAN_REPORT_PATH)
files.download(RAW_REPORT_PATH)
# 필요 시 CSV도:
# files.download(CLEAN_DATA_CSV)

엑셀 파일(.xlsx) 1개 이상 업로드하세요.


Saving 202501_04.xlsx to 202501_04.xlsx
Uploaded (in-memory): 202501_04.xlsx
Output paths set:
 RAW_REPORT_PATH   = 202501_04_QA_raw.xlsx
 CLEAN_REPORT_PATH = 202501_04_QA_clean.xlsx
 CLEAN_DATA_PATH   = 202501_04_clean.xlsx
 CLEAN_DATA_CSV    = 202501_04_clean.csv
Detected time_col: Pump-Begin
Detected pm25_col: Conc(ug/m3)
Detected metal_cols (first 10): ['Cr(ng/m3)', 'Co(ng/m3)', 'Ni(ng/m3)', 'As(ng/m3)', 'Cd(ng/m3)', 'Sb(ng/m3)', 'Pb(ng/m3)']
Detected numeric_cols (first 10): ['Pump-Begin', 'Pump-End', 'MassResetTime', 'Conc(ug/m3)', 'Cr(ng/m3)', 'Co(ng/m3)', 'Ni(ng/m3)', 'As(ng/m3)', 'Cd(ng/m3)', 'Sb(ng/m3)']
=== Cleaning Summary ===
Target columns (n=8): ['Conc(ug/m3)', 'Cr(ng/m3)', 'Co(ng/m3)', 'Ni(ng/m3)', 'As(ng/m3)', 'Cd(ng/m3)'] ...
Negatives (before -> after): 1817 -> 0
IQR masked cells (k=3.0): 62
Empty-target dropped rows: 1
Shape: raw (1683, 11) -> clean (1682, 11)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [3]:
# As만 LOD/2 대체 적용

In [4]:
# =========================
# Namhae PM2.5 Metals: Cleaning vFinal (+ As LOD/2)
# 정책: 음수→NaN, IQR_k=3.0 셀-마스킹(단, As 제외), 행 삭제 금지(빈 타깃행만 삭제),
#       RAW/CLEAN QA 저장, As는 LOD/2 대체 적용
# =========================

import pandas as pd, numpy as np, io, os, re
from google.colab import files
pd.options.display.float_format = "{:.6g}".format

# ---- 설정 ----
CONFIRM_DETECTED_COLUMNS = True
NEGATIVE_TO_NAN = True
APPLY_IQR_FILTER = True
IQR_K = 3.0
DROP_EMPTY_TARGET = True

# === As LOD/2 설정(필수: 실험실 LOD로 교체) ===
AS_COL = "As(ng/m3)"
AS_LOD = 0.05            # ← 실험 LOD 값으로 수정
USE_AS_LOD_THRESHOLD = True   # True면 (< LOD)도 ND로 간주해 LOD/2 대체

# ---- 유틸 ----
def _safe_basename(name: str):
    base, ext = os.path.splitext(name)
    base = re.sub(r"[^\w\-]+", "_", base).strip("_")
    return base, ext

def detect_columns(df: pd.DataFrame):
    df = df.copy(); df.columns = [str(c).strip() for c in df.columns]
    # 시간/PM2.5
    time_candidates = [c for c in df.columns if any(k in c.lower() for k in ["pump-begin","date","datetime","time","측정일시"])]
    pm25_candidates = [c for c in df.columns if ("pm2.5" in c.lower()) or ("con(ug/m3)" in c.lower()) or ("ug/m3" in c.lower() and "con" in c.lower())]
    time_col = time_candidates[0] if time_candidates else None
    pm25_col = pm25_candidates[0] if pm25_candidates else None
    # 금속 단위 패턴
    metal_markers = ["(ng/m3)", "(ng/㎥)", "ng/m3", "ng/㎥"]
    metal_cols = [c for c in df.columns if any(m in c.lower() for m in metal_markers)]
    return {"time_col": time_col, "pm25_col": pm25_col, "metal_cols": metal_cols}

def summarize_series(s: pd.Series, name: str):
    s_num = pd.to_numeric(s, errors="coerce")
    return pd.Series({
        "col": name,
        "count_total": int(s_num.shape[0]),
        "count_valid": int(s_num.notna().sum()),
        "valid_pct": float(100.0 * s_num.notna().mean()),
        "mean": float(s_num.mean(skipna=True)) if s_num.notna().any() else np.nan,
        "std": float(s_num.std(skipna=True)) if s_num.notna().any() else np.nan,
        "p5": float(s_num.quantile(0.05)) if s_num.notna().any() else np.nan,
        "p50": float(s_num.quantile(0.50)) if s_num.notna().any() else np.nan,
        "p95": float(s_num.quantile(0.95)) if s_num.notna().any() else np.nan,
        "min": float(s_num.min(skipna=True)) if s_num.notna().any() else np.nan,
        "max": float(s_num.max(skipna=True)) if s_num.notna().any() else np.nan,
    })

def build_qc_table(df: pd.DataFrame, targets: list):
    return pd.DataFrame([summarize_series(df[c], c) for c in targets if c in df.columns])

# ---- 업로드 & 경로 ----
print("엑셀 파일(.xlsx) 1개 업로드하세요.")
uploaded = files.upload()
if not uploaded:
    raise RuntimeError("업로드된 파일이 없습니다.")

xlsx_names = [k for k in uploaded.keys() if k.lower().endswith(".xlsx")]
src_name = xlsx_names[0]
_base, _ = _safe_basename(src_name)
RAW_REPORT_PATH   = f"{_base}_QA_raw.xlsx"
CLEAN_REPORT_PATH = f"{_base}_QA_clean.xlsx"
CLEAN_DATA_PATH   = f"{_base}_clean.xlsx"
CLEAN_DATA_CSV    = f"{_base}_clean.csv"

df_raw = pd.read_excel(io.BytesIO(uploaded[src_name]))
df_raw.columns = [str(c).strip() for c in df_raw.columns]

# ---- 컬럼 자동검출 ----
det = detect_columns(df_raw)
time_col, pm25_col, metal_cols = det["time_col"], det["pm25_col"], det["metal_cols"]

if not CONFIRM_DETECTED_COLUMNS:
    raise RuntimeError("자동 검출된 컬럼 확인 후 True로 전환하세요.")

targets = ([pm25_col] if pm25_col else []) + [c for c in metal_cols if c]
targets = [c for c in targets if c in df_raw.columns]
assert len(targets) > 0, "PM2.5/금속 대상 컬럼을 찾지 못했습니다."

# ---- QA: RAW ----
qc_raw = build_qc_table(df_raw, targets)
with pd.ExcelWriter(RAW_REPORT_PATH, engine="openpyxl") as w:
    qc_raw.to_excel(w, index=False, sheet_name="RAW_QA")

# ---- 클리닝 ----
df_clean = df_raw.copy()
if (time_col is not None) and (time_col in df_clean.columns):
    # 엑셀 시리얼 대비 안전 파싱
    s = df_clean[time_col]
    if np.issubdtype(s.dtype, np.number):
        df_clean[time_col] = pd.to_datetime(s, unit='D', origin='1899-12-30', errors='coerce')
    else:
        df_clean[time_col] = pd.to_datetime(s, errors='coerce')
    df_clean = df_clean.sort_values(time_col).reset_index(drop=True)

num = df_clean[targets].apply(pd.to_numeric, errors="coerce")

# 음수 → NaN
neg_before = int((num < 0).sum().sum())
if NEGATIVE_TO_NAN:
    num = num.mask(num < 0, np.nan)

# === As LOD/2 대체 (IQR 전에 적용) ===
lod_subs = {c: 0 for c in targets}
if AS_COL in num.columns:
    as_series = num[AS_COL].copy()
    if USE_AS_LOD_THRESHOLD:
        nd_mask = as_series.isna() | (as_series <= 0) | (as_series < AS_LOD)
    else:
        nd_mask = as_series.isna() | (as_series <= 0)
    lod_subs[AS_COL] = int(nd_mask.sum())
    num.loc[nd_mask, AS_COL] = AS_LOD / 2.0

# IQR_k=3.0 셀-마스킹 (단, As 제외)
cells_masked = 0
mask_counts = {c: 0 for c in targets}
if APPLY_IQR_FILTER:
    Q1 = num.quantile(0.25)
    Q3 = num.quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - IQR_K * IQR
    upper = Q3 + IQR_K * IQR

    out_mask = num.lt(lower, axis=1) | num.gt(upper, axis=1)
    if AS_COL in out_mask.columns:
        out_mask[AS_COL] = False  # As는 IQR 제외
    for c in targets:
        mask_counts[c] = int(out_mask[c].sum(skipna=True))
    cells_masked = int(out_mask.sum().sum())
    num = num.mask(out_mask, np.nan)

# 적용
df_clean[targets] = num

# 대상 전부 NaN 행만 삭제
rows_before = len(df_clean)
if DROP_EMPTY_TARGET:
    df_clean = df_clean.dropna(subset=targets, how="all").reset_index(drop=True)
rows_dropped_empty = rows_before - len(df_clean)

neg_after = int((df_clean[targets].apply(pd.to_numeric, errors="coerce") < 0).sum().sum())

# ---- 요약 출력 ----
print("=== Cleaning Summary (vFinal + As LOD/2) ===")
print(f"Targets (n={len(targets)}): {targets[:6]}{' ...' if len(targets)>6 else ''}")
print(f"Negatives (before -> after): {neg_before} -> {neg_after}")
print(f"IQR masked cells (k={IQR_K}): {cells_masked}  | by col: {mask_counts}")
print(f"As LOD/2 substitutions: {lod_subs.get(AS_COL, 0)} (LOD={AS_LOD})")
print(f"Empty-target dropped rows: {rows_dropped_empty}")
print(f"Shape: raw {df_raw.shape} -> clean {df_clean.shape}")

# ---- QA: CLEAN (LOD/IQR 카운트 포함 보고) ----
qc_clean = build_qc_table(df_clean, targets)
# 보조 메타정보 시트 생성
with pd.ExcelWriter(CLEAN_REPORT_PATH, engine="openpyxl") as w:
    qc_clean.to_excel(w, index=False, sheet_name="CLEAN_QA")
    # LOD/IQR 집계 시트
    meta = pd.DataFrame({
        "col": targets,
        "lod_subs": [lod_subs.get(c, 0) for c in targets],
        "iqr_masked": [mask_counts.get(c, 0) for c in targets]
    })
    meta.to_excel(w, index=False, sheet_name="ACTIONS")

# ---- 저장 ----
df_clean.to_excel(CLEAN_DATA_PATH, index=False)
df_clean.to_csv(CLEAN_DATA_CSV, index=False)

files.download(CLEAN_DATA_PATH)
files.download(CLEAN_REPORT_PATH)
files.download(RAW_REPORT_PATH)
# files.download(CLEAN_DATA_CSV)

엑셀 파일(.xlsx) 1개 업로드하세요.


Saving 202501_04.xlsx to 202501_04 (1).xlsx
=== Cleaning Summary (vFinal + As LOD/2) ===
Targets (n=8): ['Conc(ug/m3)', 'Cr(ng/m3)', 'Co(ng/m3)', 'Ni(ng/m3)', 'As(ng/m3)', 'Cd(ng/m3)'] ...
Negatives (before -> after): 1817 -> 0
IQR masked cells (k=3.0): 36  | by col: {'Conc(ug/m3)': 0, 'Cr(ng/m3)': 0, 'Co(ng/m3)': 19, 'Ni(ng/m3)': 0, 'As(ng/m3)': 0, 'Cd(ng/m3)': 0, 'Sb(ng/m3)': 0, 'Pb(ng/m3)': 17}
As LOD/2 substitutions: 1657 (LOD=0.05)
Empty-target dropped rows: 0
Shape: raw (1683, 11) -> clean (1683, 11)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>